In [ ]:
#!pip install numpy
#!pip install --user -U nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
#!pip install unidecode
#!pip install matplotlib
#!pip install pyhive
#!pip install seaborn

In [1]:
import pandas as pd
import nltk
import time
import datetime
import boto3
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import RSLPStemmer
from unidecode import unidecode
from pyhive import hive
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

warnings.filterwarnings("ignore")

In [2]:
#Carrega dados refined do Hive
#conn = hive.Connection(host=host,port= 10000)
#dataframe = pd.read_sql("SELECT wiki_id, title, wiki_timestamp, wiki_user, bot FROM defined.wikidata", conn)

In [3]:
#Carrega dados raw do csv
df_full = pd.read_csv("../../s3_data/raw/mergedfile_v3.csv", names=["schema", "wiki_id", "type", "namespace", "title", "comment", "timestamp", "user", "bot", "minor", "patrolled", "server_url", "server_name", "server_script_path", "wiki", "parsedcomment"])

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df = df_full[["wiki_id", "title", "timestamp", "user","bot"]]

In [5]:
df.head()

,wiki_id,title,timestamp,user,bot
0,96930022,'Categoria:!Esboços maiores que 1000 bytes',1587209351,'170.79.220.123',False
1,96930023,'Categoria:!Esboços menores que 1001 bytes',1587209351,'170.79.220.123',False
2,96940639,'Categoria:!Páginas com ficheiros inexistentes',1587241013,'179.124.221.246',False
3,96940640,'Categoria:Nascidos em 2000',1587241013,'179.124.221.246',False
4,96940638,'Categoria:Nascidos em 2002',1587241013,'179.124.221.246',False


## Parte 1
## Pré Processamento

In [6]:
#Remove linhas com dados nulos, transformando em minusculo e removendo aspas simples
df.dropna(inplace=True)
df["user"] = df["user"].replace({'\'': ''}, regex=True)
lw_text = df["title"].str.lower()
lw_text = lw_text.replace({'\'': ''}, regex=True)
df["user"] = df["user"].str.strip().replace({'^[0-9]*$': 'unknown'}, regex=True)

<ipython-input-6-591ba58e7dd9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-6-591ba58e7dd9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["user"] = df["user"].replace({'\'': ''}, regex=True)
<ipython-input-6-591ba58e7dd9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["u

In [7]:
#Normalizando timestamp
s = "01/01/2020"
default_timestamp = int(datetime.datetime.strptime('01/01/2020', '%d/%m/%Y').strftime("%s"))
df["timestamp"] = df["timestamp"].str.strip().replace({'^((?![0-9]).)*$': default_timestamp}, regex=True)

<ipython-input-7-65143d4881b5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["timestamp"] = df["timestamp"].str.strip().replace({'^((?![0-9]).)*$': default_timestamp}, regex=True)


In [8]:
#Substitui valores diferentes de booleano pela item de maior frequencia
max_freq = df.bot.mode()[0]
df["bot"] = df["bot"].str.strip().replace({'^((?!(False|True)).)*$': max_freq.strip()}, regex=True)

AttributeError: Can only use .str accessor with string values!

In [9]:
#Cria os tokens dos titulos
tokens =  lw_text.apply(word_tokenize)

In [10]:
#Normalizando com unicode
tokens_uni = tokens.apply(lambda x: [unidecode(z) for z in x ])

In [11]:
#Remove stopwords
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords.extend(['categoria','artigos', 'predefinicao','ficheiro','sobre','predefinicoes','redirecionamentos','esbocos','ligados','elemento','inexistentes','ficheiros','usuario','wikidata','paginas','wikipedia','discussao','lista'])
stopwords = set(stopwords + list(punctuation))
title_cleaned = tokens_uni.apply(lambda line:  [w for w in line if not w in stopwords])

In [12]:
#Cria coluna com os titulos tratados
df["title_cleaned"] = title_cleaned.apply(lambda line: " ".join(line))
df.replace("", np.nan, inplace=True)
df.dropna(inplace=True)
df.head()

<ipython-input-12-45cc9113a9d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title_cleaned"] = title_cleaned.apply(lambda line: " ".join(line))
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
<ipython-input-12-45cc9113a9d9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inp

,wiki_id,title,timestamp,user,bot,title_cleaned
131072,97475656,Faxinildo,1589310939,Fã dos sites,False,faxinildo
131073,97475664,Rumours,1589310945,Leandro Drudo,False,rumours
131074,97475665,Petrix Barbosa,1589310945,79a,False,petrix barbosa
131075,97475666,Usuário(a) Discussão:Alexromeu,1589310955,79a,False,alexromeu
131076,97475667,Usuário(a):The WWE man,1589310962,The WWE man,False,the wwe man


In [13]:
#Cria coluna com os titulos com steamming
def Stemming(sentence):
    stemmer = RSLPStemmer()
    phrase = []
    for word in sentence:
        phrase.append(stemmer.stem(word.lower()))
    return phrase

stemmed_list = title_cleaned.apply(lambda line: Stemming(line))
df["title_stemmed"] = stemmed_list.apply(lambda line: " ".join(line))
df.head()

<ipython-input-13-b3587fceacc2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title_stemmed"] = stemmed_list.apply(lambda line: " ".join(line))


,wiki_id,title,timestamp,user,bot,title_cleaned,title_stemmed
131072,97475656,Faxinildo,1589310939,Fã dos sites,False,faxinildo,faxinild
131073,97475664,Rumours,1589310945,Leandro Drudo,False,rumours,rumour
131074,97475665,Petrix Barbosa,1589310945,79a,False,petrix barbosa,petrix barb
131075,97475666,Usuário(a) Discussão:Alexromeu,1589310955,79a,False,alexromeu,alexrom
131076,97475667,Usuário(a):The WWE man,1589310962,The WWE man,False,the wwe man,the wwe man


In [15]:
#Cria coluna com os titulos com steamming
from nltk.stem import WordNetLemmatizer
def Lemmatizing(sentence):
    lemmatizer = WordNetLemmatizer()
    phrase = []
    for word in sentence:
        phrase.append(lemmatizer.lemmatize(word.lower()))
    return phrase

lemmatized_list = title_cleaned.apply(lambda line: Lemmatizing(line))
df["title_lemmatized"] = lemmatized_list.apply(lambda line: " ".join(line))
df.head()

<ipython-input-15-3f06dca6a78d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title_lemmatized"] = lemmatized_list.apply(lambda line: " ".join(line))


,wiki_id,title,timestamp,user,bot,title_cleaned,title_stemmed,title_lemmatized
131072,97475656,Faxinildo,1589310939,Fã dos sites,False,faxinildo,faxinild,faxinildo
131073,97475664,Rumours,1589310945,Leandro Drudo,False,rumours,rumour,rumour
131074,97475665,Petrix Barbosa,1589310945,79a,False,petrix barbosa,petrix barb,petrix barbosa
131075,97475666,Usuário(a) Discussão:Alexromeu,1589310955,79a,False,alexromeu,alexrom,alexromeu
131076,97475667,Usuário(a):The WWE man,1589310962,The WWE man,False,the wwe man,the wwe man,the wwe man
